# Calculating Ignition Delay Time (In Progress)
*Daniel I Pineda, James L Urban, and Carlos Fernandez-Pello*, 2017

### Ignition Delay Time
In this example, we go over the concept of ignition delay time, defined here as the time it takes for a combustible mixture to increase in temperature by 400 K. Different fuels display different ignition behavior depending on the temperature, pressure, and equivalence ratio. Unlike adiabatic flame temperature, chemical reactions and chemical reaction rates are relevant here; a chemical mechanism is a "map of possible pathways" from reactants to products. This code has been adapted from an [example reactor code](http://www.cantera.org/docs/sphinx/html/cython/examples/reactors_reactor1.html) available at the Cantera website. 

### Import Packages
- [cantera](http://www.cantera.org/docs/sphinx/html/index.html): A Combustion Chemistry Python Module 
    - Chemical Equilibrium
    - 1-D Flame Speed
    - Surface Chemistry
- [numpy](http://www.numpy.org/): A Mathematical Function Libary (similiar functions to MATLAB) 
    - Linear Algegra
    - Root Finding
    - Simple Data Analysis
- [matplotlib](http://matplotlib.org/): A Python Figure making and Plotting Software 

In [1]:
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

### Intitialize Gas Object
For this example we will initialize a gas object ("`igGas`") using the [GRI-MECH 3.0 mechanism](http://combustion.berkeley.edu/gri-mech/version30/text30.html), which is a chemical kinetic mechanism optimized for natural gas combustion. 

In [2]:
igGas = ct.Solution('gri30.xml')

### Set Initial Gas Mixture and Conditions
As a first example, we will first examine a mixture of methane and oxygen in argon with the following conditions:
- Composition: Stoichiometric methane ($CH_4$) and oxygen ($O_2$) with argon ($Ar$) as the bath gas. Ignition delay experiments are often carried out in an inert bath gas to isolate specific reactions and better examine the kinetics. 
- $T = 1500~K$
- $P = 101.325~kPa$

In [3]:
Tinit_ig = 1500 # Kelvin
Pinit_ig = 111325 # Pascals
igGas.TPX = Tinit_ig, Pinit_ig, "CH4:3.3,O2:6.7,AR:90"

### Set up a reactor object and reactor network
In Cantera, there are [different classes of zero-dimensional reactors depending on the system conditions you want to simulate](http://www.cantera.org/docs/sphinx/html/cython/zerodim.html). These reactors keep track of the temperature, pressure, species mole fractions, and other conditions inside of them. They can have surfaces, inlets and outlets, and moveable walls. For this example, we assume an adiabatic constant pressure reactor with an ideal gas equation of state.

Reactors can be placed in reactor networks on their own or with other reactors, and these networks can be advanced in time to run kinetics simulations.  

We initialize the reactor `igReactor` with the `igGas` we previously initialized in the last step, and we place `igReactor` in the reactor network `igReactorNetwork`. After that, we specify some variables to keep track of `igReactorNetwork` and species mole fractions as we advance the simulation time. We initialize `time` at `0.0` seconds and specify the maximum number of steps we want the simulation to take with `numSteps`. Then we initialize the variables `times` and `data` to keep track of the simulation time, temperature, and select species concentrations for all of these steps. Lastly, we initialize the variable where ignition delay time will be stored, `tIgn` as zero. 

In [4]:
igReactor = ct.IdealGasConstPressureReactor(igGas)
igReactorNetwork = ct.ReactorNet([igReactor])
time = 0.0
numSteps = 100000
times = np.zeros(numSteps)
data = np.zeros((numSteps, 11))
tIgn = 0

### Advance the simulation in a `for` loop
Here, we advance the simualtion according to a fixed time step of `1.e-7` and use the `advance` function of the `ReactorNet` class to do so. At each timestep, the simulation time, temperature, and species mole fractions are recorded into the variables we initialized in the last step. Every timestep, we check to see if the reactor temperature has increased by 400 K. If it has, we change the assignment of `tIgn` to a value other than zero. Thanks to the equality check in the conditional statement, `tIgn` will not be overwritten in subsequent timesteps. 

In [6]:
for n in range(numSteps):
    time += 1.e-7
    igReactorNetwork.advance(time)
    times[n] = time * 1e6 # time in us
    data[n,0] = igReactor.T # reactor temperature in K
    data[n,1:] = \
        igReactor.thermo['OH', 'H', 'O','H2','CH3', 'CH4', 'O2','H2O','CO','CO2'].X # species mole fractions
    if ((igReactor.T - Tinit_ig) > 400) and (tIgn == 0): # check to see if temperature has increased by 400 K
        tIgn = time*1e3 
        print("Calculated Ignition Delay is = " + str(tIgn) + " ms")
    if ((time == 1.e-7*numSteps) and (tIgn == 0)):
        print("Mixture did not ignite in the simulation time alotted.")

#np.savetxt("data.csv", data, delimiter=",")

Calculated Ignition Delay is = 3.0369 ms
